# CMB-ML Framework: Pipeline Code

# Introduction

CMB-ML manages a complex pipeline that processes data across multiple stages. Each stage produces outputs that need to be tracked, reused, and processed in later stages. Without a clear framework, this can lead to disorganized code, redundant logic, and errors.

The CMB-ML library provides a set of tools to manage these pipelines in a modular way. Each tool focuses on a specific task, such as handling data files, managing file paths, or defining pipeline stages. Together, they simplify building and maintaining complex workflows.

In the [previous notebook](./E_CMB_ML_framework.ipynb), I built up to a single stage of a pipeline. In this notebook, I look at how the pipeline can be assembled.

## Contents

View this notebook with [nbviewer](https://nbviewer.org/github/CMB-ML/cmb-ml/tree/main/demonstrations/F_CMB_ML_pipeline.ipynb#Introduction) (or in your IDE) to enable these links.

This notebook continues with the running [Example](#Example) from the previous notebook. It introduces two components of the CMB-ML library that exist outside the context of an individual stage, managing the whole pipeline instead:
- [PipelineContext](#PipelineContext): Manages the Executors in a pipeline
- [LogMaker](#LogMaker): Sets aside information used throughout the pipeline

A short [Conclusion](#Conclusion) wraps up and provides a segue to the next notebook. This notebook is mercifully shorter.

## Example

This notebook continues with considering the same simple task: converting a power spectrum to a map.

There are two stages. The first writes a power spectrum to the dataset as a text file. The second reads that power spectrum data and produces a "CMB" map.

(In the previous notebook, you may not have noticed the first stage running, because it was hidden in the "helper module." Almost all the code for this notebook is included.)

I do recommend looking through the Set-Up section, as some minor changes have been made to the Executors.

There is still a helper module, but it only contains import statements for use by the LogMaker object (the LogMaker was written with the assumption it would be called from a module).

## Set-up

In [ ]:
import os

# Set the location of the data directory
os.environ["CMB_ML_DATA"] = "/data/jim/CMB_Data"

In [2]:
import logging
from hydra import compose, initialize
import numpy as np
import healpy as hp

from cmbml.core import BaseStageExecutor, Asset
from cmbml.core.asset_handlers import TextPowerSpectrum, HealpyMap

I'll use a logger this time, instead of `print()`.

In [3]:
logger = logging.getLogger("F_Tutorial")
logger.setLevel(logging.DEBUG)

# Outside of a notebook, Hydra will handle the logging. 
handler = logging.StreamHandler()  # StreamHandler sends logs to sys.stdout by default
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

These are the executors used previously.

In [4]:
class MakePSExecutor(BaseStageExecutor):
    def __init__(self, cfg):
        super().__init__(cfg, stage_str="ps_setup")

        self.out_cmb_ps: Asset = self.assets_out["cmb_ps"]
        # I note the handler for assets. When using an IDE, this makes
        #   it easier to navigate to the handler's code.
        out_cmb_ps_handler: TextPowerSpectrum

    def execute(self):
        ell = np.arange(200)
        # This is a naive model for the CMB power spectrum.
        #   It's just a polynomial fit up to ell=200 for the 
        #   Planck 2018 power spectrum.
        cheap_model = [ 1.51935454e-13,
                       -1.33044280e-10,
                        4.87473463e-08,
                       -9.68198860e-06,
                        1.12257186e-03, 
                       -7.62816561e-02,
                        3.00276536e+00, 
                       -4.49411282e+01,
                        1.04893659e+03]
        ps = np.poly1d(cheap_model)
        self.out_cmb_ps.write(data=ps(ell))
        logger.info(f"CMB power spectrum written to {self.out_cmb_ps.path}")

In [5]:
class PS2MapExecutor(BaseStageExecutor):
    def __init__(self, cfg) -> None:
        super().__init__(cfg, stage_str="ps2map")

        self.out_map_asset = self.assets_out["cmb_map"]
        self.in_ps_asset = self.assets_in["cmb_ps"]
        # Note handlers for easy reference
        out_map_handler: HealpyMap
        in_ps_handler: TextPowerSpectrum

        self.nside = cfg.scenario.nside

    def execute(self) -> None:
        logger.info(f"Power spectrum read from {self.in_ps_asset.path}")
        ps = self.in_ps_asset.read()
        cmb = hp.synfast(ps, nside=self.nside)
        self.out_map_asset.write(data=cmb)
        logger.info(f"Map written to {self.out_map_asset.path}")
        return

**Note**: In a top-level script, I would instead have something like:

```python
from phase_dir.stage_executors import (
    MakePSExecutor,
    PS2MapExecutor
    )
```

Here, I just include the code. Take a look at [the top-level simulation script](../main_sims.py) for an example of these imports. 

I also need the configuration.

In [6]:
from omegaconf import OmegaConf
from hydra.core.hydra_config import HydraConfig

# I use a context here; `initialize` can be used in a couple different ways.
with initialize(version_base=None, config_path="cfg"):
    # Use return_hydra_config=True to get the 'hydra' information in the config object
    cfg = compose(config_name="config_demoF_pipeline", return_hydra_config=True)
    # Because initialize was used, HydraConfig was not properly configured.
    # It requires that 'hydra' is in the config object.
    # This is a workaround for this demo notebook and need not be used elsewhere.
    HydraConfig.instance().set_config(cfg)

# PipelineContext

I use top-level scripts to run several stages of a pipeline. Those scripts cover multiple stages within a phase (such as all the stages for producing a simulation). Certain practicalities make it so that the stages aren't delineated as neatly as in some of my pipeline diagrams.

Running multiple Executors in a row is handled by a single `PipelineContext` manager. That object has a few jobs:
- It sets up Executors to run in order
- It runs the Executors's `execute()` methods
- It enables some preliminary checks for errors

In [7]:
from cmbml.core import PipelineContext

In [8]:
pipeline_context = PipelineContext(cfg)

pipeline_context.add_pipe(MakePSExecutor)
pipeline_context.add_pipe(PS2MapExecutor)

The `prerun_pipeline()` method goes through the list of Executors and initializes each one. This way I can start a pipeline and have some reassurance that there are no issues with the configurations of each. This informs what I put into the Executor's `__init__()` and what I put into `execute()`.

In [9]:
pipeline_context.prerun_pipeline()

Now I run the pipeline itself. I enclose it in a `try` block so that I'm assured that I don't lose valuable debugging information if something fails while executing.

In [10]:
try:
    pipeline_context.run_pipeline()
except Exception as e:
    # I typically use the logging library for these messages
    logger.warning("An exception occured during the pipeline.", exc_info=e)
    raise e
finally:
    logger.info("Pipeline completed.")

CMB power spectrum written to /data/jim/CMB_Data/Datasets/DemoNotebook/A_PS_Setup/cmb_dummy_ps.fits
Skipping stage logs for stage MakePSExecutor.
Power spectrum read from /data/jim/CMB_Data/Datasets/DemoNotebook/A_PS_Setup/cmb_dummy_ps.fits
Map written to /data/jim/CMB_Data/Datasets/DemoNotebook/B_CMB_Map/cmb_dummy_map.fits
Pipeline completed.


# LogMaker

As much as I have error-checks and guard rails to prevent issues, those still arise and may do so silently. For this reason, I'm very careful to keep track of what was run. An early version of CMB-ML used a system called DVC to enable this, but I found it difficult to use (likely due to my own inflexibility!). To this end, I have a `LogMaker`. For each stage of the pipeline, it logs:
- The configurations used, in both raw form and as interpolated by Hydra
- All CMB-ML python modules imported (but not modules from external libraries)
- The `logging` output for stages run

Similar to the Namer, this runs in the background and I seldom think about it. The only place it is used directly is the top-level script. I repeat most of what I had in the PipelineContext to give the following example of it being used. 

**Note**: Currently this doesn't work fully in the notebook; there's a conflict in building the configuration I haven't been able to resolve. The Dataset folder will contain only configuration logs, without Python. This works well outside the notebook and a top-level script should be run if you're interested in a demonstration.

In [11]:
from pathlib import Path
from cmbml.core import LogMaker


# Initialize the log maker
log_maker = LogMaker(cfg)
# Log the procedure to the hydra log (getting python modules)
# I use a dummy module here. 
helper_dir = Path(os.path.abspath(os.getcwd())) / "helpers"
log_maker.log_procedure_to_hydra(helper_dir / "F_helper.py")
# Generally, I'd use __file__, as in the following, but that fails in notebooks.
# log_maker.log_procedure_to_hydra(source_script=__file__)

# Create the pipeline context, with the log_maker, so each stage can log
pipeline_context = PipelineContext(cfg, log_maker)

pipeline_context.add_pipe(MakePSExecutor)
pipeline_context.add_pipe(PS2MapExecutor)

pipeline_context.prerun_pipeline()

try:
    pipeline_context.run_pipeline()
except Exception as e:
    print("An exception occured during the pipeline.", exc_info=e)
    raise e
finally:
    # Add this line to the end of the pipeline, keeping logs with the dataset
    log_maker.copy_hydra_run_to_dataset_log()
    print("Pipeline completed.")

CMB power spectrum written to /data/jim/CMB_Data/Datasets/DemoNotebook/A_PS_Setup/cmb_dummy_ps.fits
Skipping stage logs for stage MakePSExecutor.
Power spectrum read from /data/jim/CMB_Data/Datasets/DemoNotebook/A_PS_Setup/cmb_dummy_ps.fits
Map written to /data/jim/CMB_Data/Datasets/DemoNotebook/B_CMB_Map/cmb_dummy_map.fits


Pipeline completed.


Notice the message "Skipping stage logs for stage MakePSExecutor." If I look at the config:

In [12]:
print(OmegaConf.to_yaml(cfg.pipeline))

ps_setup:
  assets_out:
    cmb_ps:
      handler: TextPowerSpectrum
      path_template: '{root}/{dataset}/{stage}/cmb_dummy_ps.fits'
  dir_name: A_PS_Setup
ps2map:
  assets_out:
    cmb_map:
      handler: HealpyMap
      path_template: '{root}/{dataset}/{stage}/cmb_dummy_map.fits'
  assets_in:
    cmb_ps:
      stage: ps_setup
  dir_name: B_CMB_Map
  make_stage_log: true



I see that the `ps2map` stage has `make_stage_log: true`, but `ps_setup` is missing that key:value pair. When `make_stage_log: true`, the LogMaker puts the contents of the Logs into the directory associated with the output of that particular stage. If it is missing or `false`, this does not happen and a small warning is given.

For the long-running stages that produce a large volume of output, I sometimes transfer these directories and want to keep the Logs with them by default. The logs are a relatively small amount of data has been a worthwhile cost. Hoever, for many of the smaller stages (especially in the Analysis phase which outputs a bunch of figures), the Logs are unnecessary. It is for this reason that they can be disabled.

# Conclusion

This has been an overview of how stages are assembled and run in bulk. 

- The `PipelineContext` class is used to collect stages of a pipeline, check that they will be properly initialized, and then run the stages in order
- The `LogMaker` class is used to ensure reproducibility, creating an image of the codebase as it existed when the data was actually processed.

These tools are important when setting up a top-level script. I hope that it explains some design decisions made in the structure of CMB-ML.

In the [next notebook](./G_CMB_ML_executors.ipynb), I'll take a closer look at a few different patterns for Executors.